In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

## Create folders for keypoints

In [ ]:
for i in os.listdir("Path/to/directory/containing/videos"):
    os.mkdir("Path/to/directory/containing/keypoints/" + i)
    for j in os.listdir("Path/to/directory/containing/videos/" + i):
        os.mkdir("Path/to/directory/containing/keypoints/" + i + "/" + j.split(".")[0])

## Keypoints detection for every frame using mediapipe

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
def mediapipe_detection(image, model):
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

## Draw keypoints and connecting lines

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

## Keypoints extraction

In [ ]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

## Extraction process

In [ ]:
for i in os.listdir("Path/to/directory/containing/videos"): 
    for j in os.listdir("Path/to/directory/containing/videos/" + i):
        try:
            cap = cv2.VideoCapture("Path/to/directory/containing/videos/" + i + "/" + j)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            # Set mediapipe model 
            with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                        # Loop through video length aka sequence length
                for frame_num in range(30): # number of frames

                    # Read feed
                    ret, frame = cap.read()

                    # Make detections
                    image, results = mediapipe_detection(frame, holistic)

                    # Draw landmarks
                    draw_styled_landmarks(image, results)

                    # NEW Export keypoints
                    keypoints = extract_keypoints(results)
                    npy_path = "Path/to/directory/containing/keypoints/" + i + "/" + j.split(".")[0] + "/" + str(frame_num) 
                    np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

                cap.release()
                cv2.destroyAllWindows()
        except Exception as err:
            print(err)
            print("Path/to/directory/containing/videos/" + i + "/" + j)
            continue